## Gemma - First try

In [ ]:
!pip install -U transformers accelerate bitsandbytes

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from google.colab import userdata

# Get token from Colab Secrets
hf_token = userdata.get('HF_TOKEN')

model_id = "google/gemma-2b-it" # 'it' stands for instruction-tuned

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    token=hf_token
)

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [4]:
def chat_with_gemma(prompt, history=[]):
    # Add the new user message to history
    history.append({"role": "user", "content": prompt})

    # NEW CODE (Fixed)
    # 1. Generate the formatted tokens
    inputs = tokenizer.apply_chat_template(
        history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 2. Pass only the input_ids to the generator
    # Using **inputs unpacks both 'input_ids' and 'attention_mask'
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id # Added for stability
    )

    # Decode only the newly generated text
    response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)

    # Add model response to history for context
    history.append({"role": "model", "content": response})
    return response, history

In [5]:
chat_history = []
print("Gemma Bot: Hello! Type 'exit' to stop.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit': break

    response, chat_history = chat_with_gemma(user_input, chat_history)
    print(f"Gemma: {response}")

Gemma Bot: Hello! Type 'exit' to stop.
You: salam
Gemma: Wa alaikumussalam wa rahmatullahi wa barakatuh. How may I assist you today?
You: سلام
Gemma: Wa salam wa rahmmatullahi wa barakatuh. How can I assist you today?
You: فارسی می تونی حرف بزنی
Gemma: No problem! I can help you with any questions or tasks you may have. What can I do for you?
You: پایتون چیه
Gemma: پایتون، من را بخواشت که دلیمش به service serve ملحوظی بتون معطوف.
You: bye
Gemma: Nailed it! I'm happy to assist you. How can I help?
You: exit


## Qwen - First Try

In [ ]:
!pip install -U transformers accelerate bitsandbytes

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "Qwen/Qwen2.5-7B-Instruct"

# 4-bit configuration to keep it fast and fit in VRAM
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [10]:
def chat_with_qwen(prompt, history=[]):
    # System prompt ensures it stays in Persian mode
    if not history:
        history.append({"role": "system", "content": "You are a helpful assistant. Please respond in Persian (Farsi)."})

    history.append({"role": "user", "content": prompt})

    # NEW CODE (Fixed)
    # 1. Generate the formatted tokens
    inputs = tokenizer.apply_chat_template(
        history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 2. Pass the unpacked dictionary to the generator
    # Using **inputs sends both 'input_ids' and 'attention_mask' correctly
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id # Added for stability
    )

    response = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    history.append({"role": "assistant", "content": response})
    return response, history

In [11]:
chat_history = []
print("Qwen Bot: سلام! چطور می‌توانم به شما کمک کنم؟ (برای خروج 'exit' بنویسید)")

while True:
    user_input = input("شما: ")
    if user_input.lower() in ['exit', 'خروج']: break

    response, chat_history = chat_with_qwen(user_input, chat_history)
    print(f"کوین: {response}")

Qwen Bot: سلام! چطور می‌توانم به شما کمک کنم؟ (برای خروج 'exit' بنویسید)
شما: سلام در یک خط بهم بگو پایتون چیه
کوین: پایتون یک زبان برنامه‌نویسی عالی و آسان برای یادگیری است که برای توسعه نرم‌افزارها، تحلیل داده و اینچه‌ی دیگر استفاده می‌شود.
شما: تابع چی هست
کوین: تابع در پایتون (و دیگر زبان‌های برنامه‌نویسی) یک سیستم برای تغییرات جدید و تکراری انجام دادن کارها است. شما می‌توانید یک قسمت کد را که ممکن است در چند مکان مختلف نیاز به اجرای آن داشته باشید، به یک تابع بگذارید تا آن را یکبار نوشته و در مکان‌های مختلف خود را فراخوانی کنید.
شما: پایتخت ایران کجاست؟
کوین: پایتخت ایران تهران است.
شما: فاصله ایران تا پرتغال چقدر هست؟
کوین: فاصله جغرافیایی میان ایران و پرتغال به صورت غیر مستقیم است، زیرا این دو کشور در اروپا و آسیا قرار دارند. با این حال، با توجه به مسیر مسیریابی، می‌توان گفت که فاصله میان این دو کشور حدود 6,000 کیلومتر است. این فاصله معمولاً در طی راه‌های هوایی محاسبه می‌شود.
شما: خطوط هواپیمایی معروف دنیا رو بهم بگو
کوین: برخی خطوط هواپیمایی معروف دنیا عبارتند از:

1. **آیرلاین